In [ ]:
from google import genai
from google.genai import types
import os
from PIL import Image
from datetime import datetime
import sys
from dotenv import load_dotenv
import datetime
import json
import logging
from helperfunctions import *

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(filename='PromptToImages.log', level=logging.INFO)
logger.info('Started app.py at time %s', datetime.now())

# Load configurations
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
        user_input = config['user_input']
        logger.info(f"Using the following user inputs: {config}")
except FileNotFoundError:
    logger.error("config.json not found")

load_dotenv() # Load environment variables from .env file
if not os.environ.get("GEMINI_API_KEY"):
    print("Please set the GEMINI_API_KEY environment variable.")
    print("You can get one at: https://aistudio.google.com/")
    logger.error("GEMINI_API_KEY not set in environment variables.")
else:
    client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))

filepath_prefix = f"users/{user_input['username']}/projects/{user_input['project_name']}/"

In [ ]:
# create list of prompts to generate images from single user prompt
list_of_prompts = create_list_of_prompts(
    user_prompt=user_input['image_content_description'],
    client=client,
    chat_model=user_input["text_generation_model"], 
    prompt_prefix=config.get("prompt_generation_prefix", ""), 
    prompt_suffix=config.get("prompt_generation_suffix", ""),  
    max_tries=config.get("max_tries", 3), 
    num_prompts=user_input.get("num_prompts", 1))

# Write prompts to file
output_path = filepath_prefix + 'generated_prompts/prompts.txt'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, 'w', encoding='utf-8') as f:
    for prompt in list_of_prompts:
        f.write(prompt + '\n')

In [ ]:
# optimize prompts for image generation
# Read prompts from file
with open(filepath_prefix + 'generated_prompts/prompts.txt', 'r', encoding='utf-8') as f:
    list_of_prompts = [line.strip() for line in f]

optimized_image_generation_prompts = optimize_image_prompts(
    user_prompts=list_of_prompts, 
    user_image_style_description=user_input.get("image_style_description", ""),
    client=client, 
    chat_model=user_input["text_generation_model"], 
    prompt_prefix=config.get("image_optimization_prefix", ""), 
    prompt_suffix=config.get("image_optimization_suffix", ""))

# Write optimized image generation prompts to file
output_path = filepath_prefix + 'generated_prompts/optimized_prompts.txt'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, 'w', encoding='utf-8') as f:
    for prompt in optimized_image_generation_prompts:
        f.write(prompt + '\n')

In [ ]:
# Read optimized_prompts from file
with open(filepath_prefix + 'generated_prompts/optimized_prompts.txt', 'r', encoding='utf-8') as f:
    optimized_image_generation_prompts = [line.strip() for line in f]


generate_and_save_images(optimized_image_generation_prompts,
                            client=client, 
                            image_model=user_input["image_generation_model"], 
                            prompt_prefix="",
                            prompt_suffix="",
                            num_images=user_input.get("num_images_per_prompt", 1), 
                            output_folder=filepath_prefix + "generated_images"
                            )

In [ ]:
# Read optimized_prompts from file
with open(filepath_prefix + 'generated_prompts/optimized_prompts.txt', 'r', encoding='utf-8') as f:
    optimized_image_generation_prompts = [line.strip() for line in f]

descriptions = create_descriptions(optimized_image_generation_prompts,  
                                   user_input.get("image_desription_description", ""), 
                                   user_input.get("language", "english"), 
                                   client,
                                   user_input["text_generation_model"], 
                                   prompt_prefix=config.get("image_description_generation_prefix", ""), 
                                   prompt_suffix=config.get("image_description_generation_suffix", ""))

# Write descriptions to file
output_path = filepath_prefix + 'generated_prompts/descriptions.txt'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, 'w', encoding='utf-8') as f:
    for description in descriptions:
        f.write(description + '\n')

In [ ]:
logger.info('Finished app.py at time %s', datetime.now())

In [ ]:
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import glob
import os
import re

with open(filepath_prefix + 'generated_prompts/descriptions.txt', 'r', encoding='utf-8') as f:
    descriptions = [line.strip() for line in f]

# Create PDF
pdf_filename = filepath_prefix + 'AnimalsDoingThings.pdf'
c = canvas.Canvas(pdf_filename, pagesize=A4)
width, height = A4

# Get all generated images
image_files = sorted(glob.glob(filepath_prefix + 'generated_images/*.png'))

# Group images by prompt index
images_by_prompt = {}
for image_file in image_files:
    # Filename format: image-X-version-Y-timestamp.png
    # Extract X
    basename = os.path.basename(image_file)
    match = re.search(r'image-(\d+)-version', basename)
    if match:
        prompt_idx = int(match.group(1))
        if prompt_idx not in images_by_prompt:
            images_by_prompt[prompt_idx] = []
        images_by_prompt[prompt_idx].append(image_file)

# Iterate through descriptions (which correspond to prompt indices)
for idx, description in enumerate(descriptions):
    # Get images for this prompt
    # images are 1-indexed (image-1-...), descriptions are 0-indexed
    current_images = images_by_prompt.get(idx + 1, [])
    
    if not current_images:
        continue
        
    # --- Prepare Text (Description) ---
    c.setFont("Helvetica", 12)
    
    # Wrap text
    max_line_width = width - 100
    words = description.split()
    lines = []
    current_line = []
    
    for word in words:
        current_line.append(word)
        line_text = ' '.join(current_line)
        if c.stringWidth(line_text, "Helvetica", 12) > max_line_width:
            current_line.pop()
            lines.append(' '.join(current_line))
            current_line = [word]
    if current_line:
        lines.append(' '.join(current_line))
        
    # Calculate text height and position
    line_height = 15
    text_block_height = len(lines) * line_height
    text_bottom_margin = 50
    text_top_y = text_bottom_margin + text_block_height
        
    # --- Draw Images (2x2 Grid) ---
    # Define grid area
    # Images take up space from top margin down to above the text
    grid_top = height - 50
    grid_bottom = text_top_y + 20 # Padding above text
    grid_left = 50
    grid_right = width - 50
    
    grid_width = grid_right - grid_left
    grid_height = grid_top - grid_bottom
    
    # We want 2x2, so 4 cells
    cell_width = grid_width / 2
    cell_height = grid_height / 2
    
    # Limit to 4 images per page
    for img_idx, image_file in enumerate(current_images[:4]):
        row = img_idx // 2
        col = img_idx % 2
        
        # Calculate position
        # Row 0 is top, Row 1 is bottom
        x_pos = grid_left + (col * cell_width)
        y_pos = grid_top - ((row + 1) * cell_height)
        
        # Load and scale image to fit in cell with padding
        try:
            img = ImageReader(image_file)
            img_w, img_h = img.getSize()
            
            # Padding inside cell
            padding = 10
            avail_w = cell_width - 2*padding
            avail_h = cell_height - 2*padding
            
            scale = min(avail_w / img_w, avail_h / img_h)
            scaled_w = img_w * scale
            scaled_h = img_h * scale
            
            # Center in cell
            img_x = x_pos + (cell_width - scaled_w) / 2
            img_y = y_pos + (cell_height - scaled_h) / 2
            
            c.drawImage(image_file, img_x, img_y, width=scaled_w, height=scaled_h)
        except Exception as e:
            print(f"Error adding image {image_file}: {e}")

    # --- Draw Text at Bottom ---
    text_y = text_top_y
    for line in lines:
        c.drawCentredString(width / 2, text_y, line)
        text_y -= line_height

    c.showPage()

c.save()
logger.info(f'PDF created: {pdf_filename}')
print(f'PDF created: {pdf_filename}')